In [1]:
## 17 - August - 2023

In [2]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import Ridge
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

In [3]:
stats_DataFrame = pd.read_csv('../Data/stats_DataFrame_updated.csv')

In [4]:
regression_instance = Ridge(alpha=1)
split_instance = TimeSeriesSplit(n_splits=3)
sequential_feature_selector_instance = SequentialFeatureSelector(regression_instance, n_features_to_select=20,
                                                                 direction='forward', cv=split_instance, n_jobs=2)

In [5]:
target_column = 'Next_season_WAR'
text_columns = ['Name', 'Team']
meta_data_columns = ['IDfg', 'Season']

keeper_columns = stats_DataFrame.columns[~stats_DataFrame.columns.isin([target_column] + text_columns + meta_data_columns)]

In [6]:
scaler_instance = MinMaxScaler()
stats_DataFrame.loc[:, keeper_columns] = scaler_instance.fit_transform(stats_DataFrame[keeper_columns])

In [9]:
sequential_feature_selector_instance.fit(stats_DataFrame[keeper_columns], stats_DataFrame['Next_season_WAR'])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=Ridge(alpha=1), n_features_to_select=20,
                          n_jobs=2)

In [13]:
optimal_features = list(keeper_columns[sequential_feature_selector_instance.get_support()])

In [ ]:
def backtest(stats, model_instance, features, start=5, step=1):
    hmm = []
    seasons = sorted(stats['Season'].unique())
    for iteration in range(start,len(seasons),step):
        current_season = seasons[iteration]
        training_data = stats[stats['Season'] < current_season]
        testing_data = stats[stats['Season'] == current_season]
        model_instance.fit(training_data[features], training_data['Next_season_WAR'])
        model_instance.predict(testing_data[features])